# Relevant Libraries

In [1]:
# Importing Relevant Libraries
import os, os.path
import importlib
import itertools
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import qmc
import pyDOE2 as pyd
import seaborn as sns
from matplotlib.pyplot import plot, savefig
import time
import datetime 
from datetime import date
from pyomo.opt import SolverStatus, TerminationCondition
import matplotlib.ticker as mtick
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
import graphviz 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import logging

# Source: https://danshiebler.com/2016-09-14-parallel-progress-bar/
from tqdm import tqdm
import concurrent.futures
from concurrent.futures import ProcessPoolExecutor, as_completed
import sys # Python "sys" documentation: https://docs.python.org/3/library/sys.html

In [2]:
# To reset the default parameters of matplotlib:
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)

# Importing data

In [3]:
# ------------------ Parameters not imported/computed ------------------
i = ['BECCS', 'A/R', 'SCS', 'BC', 'DACCS', 'EW', 'OA', 'DOCCS']
k1 = 0 # Value of first time period
delK = 5 # Time step of optimisation (i.e., length of each period)
N = 4 # Number of segments for piecewise linear approximation
first_year = 2020 # Year corresponding to first time period
last_year = 2100 # Year corresponding to last time period
T = int(1 + (last_year-first_year)/delK) # Number of time periods evaluated, equivalent to 17 if delK = 5 (i.e., 80 years)
k = list(range(k1,T))

# Color Map for later plotting
cmap=plt.get_cmap('Set2')
color_dict = {'BECCS':cmap(0.8), 'A/R':cmap(0.1), 'SCS':cmap(0.9), 'BC':cmap(0.2), 'DACCS':cmap(0.7), 'EW':cmap(0.3), 'OA':cmap(0.4), 'DOCCS':cmap(0.5)}

## Model input data

In [4]:
# If the kernel is restarted: run any of the "With sampling step / size" cells and then run this one
#today = date.today().strftime("%d.%m.%Y")
today = "15.08.2024"
df_input = pd.read_excel("Input_Output_Files/File Name_%s.xlsx" %today, index_col ='future_id')

# To separate all parameters (in df_input) vs the ones that change across futures (in df_LHS): saving the parameters from the original file
df = pd.read_excel("PortfolioFiles/Portfolio_Files_Latest/All data Step size.xlsx", index_col = "Parameter")
units = dict(zip(df.index, df.Unit)) # Save all units in a dictionary for later use
df = df[df.minimum != df.maximum]
df = df[['minimum','maximum']]
dict_ranges = df.T.to_dict('records')[0]

# If only the LHS table is wanted:
df_LHS = df_input[list(dict_ranges.keys())].T

# If only the Y_ref is wanted:
Initial_YRef = df_input[df_input.columns.intersection(['Yref_' + el for el in i])]
Initial_YRef.columns = [str(col).split('_')[1] for col in Initial_YRef.columns]
Initial_YRef = Initial_YRef.loc[0].to_dict()

n_lhs = len(df_LHS.T.index)-1 # "-1" because we added future 0 :)

df_LHS.head()

future_id,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,3000
Energy_DACCS,4.7,4.700,10.000,6.467,10.000,6.467,4.7,8.233,4.700,6.467,...,8.233,8.233,4.700,10.000,10.000,8.233,6.467,10.000,10.000,4.7
Energy_DOCCS,2.5,8.300,8.300,2.500,6.367,8.300,8.3,6.367,2.500,6.367,...,8.300,8.300,8.300,4.433,2.500,2.500,2.500,6.367,2.500,2.5
Energy_EW,0.5,12.500,0.500,4.500,12.500,8.500,12.5,0.500,12.500,12.500,...,8.500,8.500,4.500,12.500,8.500,8.500,12.500,0.500,4.500,0.5
Energy_OA,0.7,4.767,2.733,6.800,2.733,2.733,6.8,2.733,4.767,2.733,...,4.767,6.800,4.767,2.733,4.767,2.733,0.700,4.767,2.733,0.7
EnergyL,2030.0,2045.000,2045.000,2030.000,2030.000,2045.000,2045.0,2050.000,2040.000,2050.000,...,2045.000,2030.000,2035.000,2050.000,2040.000,2030.000,2050.000,2050.000,2040.000,2040.0


In [5]:
Uncertainties = dict(zip(df_LHS.index.to_list(),['DACCS Energy Requirements','DOCCS Energy Requirements','EW Energy Requirements', 'OA Energy Requirements', 'Year at which energy becomes available for CDR',
'A/R Land Requirements','BECCS Land Requirements','Land Limit','A/R Experience Parameter','BC Experience Parameter','BECCS Experience Parameter','DACCS Experience Parameter',
'DOCCS Experience Parameter','EW Experience Parameter','OA Experience Parameter','SCS Experience Parameter','Removals Required (2020 - 2100)','A/R Maximum Potential', 'BC Maximum Potential', 'BECCS Maximum Potential',
'DACCS Maximum Potential', 'DOCCS Maximum Potential', 'EW Maximum Potential', 'OA Maximum Potential', 'SCS Maximum Potential', 'Annual Limit to Geological Injection']))
Uncertainties

{'Energy_DACCS': 'DACCS Energy Requirements',
 'Energy_DOCCS': 'DOCCS Energy Requirements',
 'Energy_EW': 'EW Energy Requirements',
 'Energy_OA': 'OA Energy Requirements',
 'EnergyL': 'Year at which energy becomes available for CDR',
 'Land_A/R': 'A/R Land Requirements',
 'Land_BECCS': 'BECCS Land Requirements',
 'LandL': 'Land Limit',
 'ER_A/R': 'A/R Experience Parameter',
 'ER_BC': 'BC Experience Parameter',
 'ER_BECCS': 'BECCS Experience Parameter',
 'ER_DACCS': 'DACCS Experience Parameter',
 'ER_DOCCS': 'DOCCS Experience Parameter',
 'ER_EW': 'EW Experience Parameter',
 'ER_OA': 'OA Experience Parameter',
 'ER_SCS': 'SCS Experience Parameter',
 'CDRRequired': 'Removals Required (2020 - 2100)',
 'Ymax_A/R': 'A/R Maximum Potential',
 'Ymax_BC': 'BC Maximum Potential',
 'Ymax_BECCS': 'BECCS Maximum Potential',
 'Ymax_DACCS': 'DACCS Maximum Potential',
 'Ymax_DOCCS': 'DOCCS Maximum Potential',
 'Ymax_EW': 'EW Maximum Potential',
 'Ymax_OA': 'OA Maximum Potential',
 'Ymax_SCS': 'SCS Max

## Model output data

In [6]:
#Reading aggregated results to csv:
#today = date.today().strftime("%d.%m.%Y")
today = "27.11.2024"

aggregated_results = pd.read_csv("Input_Output_Files/aggregated_results_%s.csv" %today, index_col ='future_id')
#results_in_future = aggregated_results[['Cumulative capacity','Annual Capacity','Removals per period','Costs per period (undiscounted)','End. Cumulative Costs']]

all_metrics_ranges = {key: [] for key in aggregated_results.columns.to_list()}
for metric in aggregated_results.columns:
    all_metrics_ranges[metric].append([aggregated_results[metric].min(),aggregated_results[metric].max()])

total_future = aggregated_results[['Removals','Costs']]
max_resources = aggregated_results[['Land','Energy','Water','Nitrogen','Phosphorous']]   
max_capacity = aggregated_results[i]

removed_per_CDR = aggregated_results[aggregated_results.columns.intersection([el+'_removed' for el in i])]
removed_per_CDR.columns = [str(col).split('_')[0] for col in removed_per_CDR.columns]

max_2050 = aggregated_results[aggregated_results.columns.intersection([el+'_%d' %2050 for el in i])].T
max_2050.index = [str(col).split('_')[0] for col in max_2050.index]

max_2075 = aggregated_results[aggregated_results.columns.intersection([el+'_%d' %2075 for el in i])].T
max_2075.index = [str(col).split('_')[0] for col in max_2075.index]

max_2100 = aggregated_results[aggregated_results.columns.intersection([el+'_%d' %2100 for el in i])].T
max_2100.index = [str(col).split('_')[0] for col in max_2100.index]

solved = aggregated_results.Solved.sum()

all_metrics_names = dict(zip(aggregated_results.columns.to_list(),['DACCS Energy Requirements','DOCCS Energy Requirements','EW Energy Requirements', 'OA Energy Requirements', 'Year at which energy becomes available for CDR',
'A/R Land Requirements','BECCS Land Requirements','Land Limit','A/R Experience Parameter','BC Experience Parameter','BECCS Experience Parameter','DACCS Experience Parameter',
'DOCCS Experience Parameter','EW Experience Parameter','OA Experience Parameter','SCS Experience Parameter','Removals Required (2020 - 2100)','A/R Maximum Potential', 'BC Maximum Potential', 'BECCS Maximum Potential',
'DACCS Maximum Potential', 'DOCCS Maximum Potential', 'EW Maximum Potential', 'OA Maximum Potential', 'SCS Maximum Potential', 'Annual Limit to Geological Injection','Solved Futures', 'Tons Removed', 'Discounted Cumulative Costs',
'Maximum Capacity of AR','Maximum Capacity of BC','Maximum Capacity of BECCS','Maximum Capacity of DACCS','Maximum Capacity of DOCCS', 'Maximum Capacity of EW', 'Maximum Capacity of OA', 'Maximum Capacity of SCS',
'Land used for CDR', 'Energy used for CDR','Water used for CDR', 'Nitrogen used for CDR', 'Phosphorous used for CDR','Maximum Capacity BECCS by 2050','Maximum Capacity A/R by 2050','Maximum Capacity SCS by 2050','Maximum Capacity BC by 2050','Maximum Capacity DACCS by 2050',
'Maximum Capacity EW by 2050','Maximum Capacity OA by 2050','Maximum Capacity DOCCS by 2050','Maximum Capacity BECCS by 2075','Maximum Capacity A/R by 2075','Maximum Capacity SCS by 2075','Maximum Capacity BC by 2075','Maximum Capacity DACCS by 2075',
'Maximum Capacity EW by 2075','Maximum Capacity OA by 2075','Maximum Capacity DOCCS by 2075','Maximum Capacity BECCS by 2100','Maximum Capacity A/R by 2100','Maximum Capacity SCS by 2100','Maximum Capacity BC by 2100','Maximum Capacity DACCS by 2100',
'Maximum Capacity EW by 2100','Maximum Capacity OA by 2100','Maximum Capacity DOCCS by 2100']))

all_metrics_units = dict(zip(aggregated_results.columns.to_list(),['GJ/tCO2','GJ/tCO2','GJ/tCO2','GJ/tCO2', 'year', 'ha/tCO2','ha/tCO2','ha',
'%','%','%','%','%','%','%','%','tCO2','tCO2','tCO2','tCO2','tCO2','tCO2','tCO2','tCO2','tCO2','tCO2/yr','binary', 'tCO2', 'USD','tCO2','tCO2','tCO2','tCO2','tCO2','tCO2','tCO2','tCO2',
'ha', 'GJ','km3', 'Mt N', 'Mt P','tCO2/yr','tCO2/yr','tCO2/yr','tCO2/yr','tCO2/yr','tCO2/yr','tCO2/yr','tCO2/yr','tCO2/yr','tCO2/yr','tCO2/yr','tCO2/yr','tCO2/yr','tCO2/yr','tCO2/yr','tCO2/yr',
'tCO2/yr','tCO2/yr','tCO2/yr','tCO2/yr','tCO2/yr','tCO2/yr','tCO2/yr','tCO2/yr']))

all_metrics_ranges = {key: [] for key in aggregated_results.columns.to_list()}
for metric in aggregated_results.columns:
    all_metrics_ranges[metric].append([aggregated_results[metric].min(),aggregated_results[metric].max()])

aggregated_results

,Energy_DACCS,Energy_DOCCS,Energy_EW,Energy_OA,EnergyL,Land_A/R,Land_BECCS,LandL,ER_A/R,ER_BC,...,OA_2100,DOCCS_2100,BECCS_removed,A/R_removed,SCS_removed,BC_removed,DACCS_removed,EW_removed,OA_removed,DOCCS_removed
future_id,,,,,,,,,,,,,,,,,,,,,
0,4.700,2.500,0.5,0.700,2030,0.07,0.040,200000000,-0.230,-0.050,...,6.982525e+08,0.0,2.730000e+07,1.500000e+11,3.300000e+10,1.050750e+10,1.725000e+11,2.000000e+10,1.396505e+10,10.0
1,4.700,8.300,12.5,4.767,2045,0.08,0.170,1000000000,-0.230,-0.033,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10.000,8.300,0.5,2.733,2045,0.08,0.170,600000000,-0.230,0.083,...,0.000000e+00,0.0,2.730000e+07,1.936790e+11,1.166000e+11,9.090750e+10,9.878603e+10,1.000000e+01,1.000000e+01,10.0
3,6.467,2.500,4.5,6.800,2030,0.07,0.170,400000000,-0.202,0.100,...,1.000000e+09,0.0,6.636388e+10,1.822000e+11,8.775000e+10,2.255070e+11,3.708050e+11,4.952163e+10,1.785220e+10,10.0
4,10.000,6.367,12.5,2.733,2030,0.07,0.040,600000000,-0.063,0.017,...,1.080000e+10,0.0,1.141960e+11,2.794000e+11,6.200000e+10,2.325750e+10,1.539640e+11,1.000000e+01,2.671830e+11,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2996,8.233,2.500,8.5,2.733,2030,0.09,0.105,700000000,-0.202,0.017,...,5.200000e+09,0.0,9.600568e+10,2.794000e+11,1.590000e+11,1.538070e+11,2.970480e+11,6.484983e+08,1.140900e+11,10.0
2997,6.467,2.500,12.5,0.700,2050,0.08,0.040,300000000,-0.174,-0.050,...,3.800000e+09,0.0,8.970591e+10,2.213270e+11,1.590000e+11,4.185750e+10,1.197780e+11,7.040105e+09,1.612920e+11,10.0
2998,10.000,6.367,0.5,4.767,2050,0.09,0.105,200000000,0.020,-0.017,...,1.220000e+10,0.0,2.730000e+07,1.625000e+11,1.000000e+11,3.900750e+10,1.487922e+10,1.081155e+10,1.727750e+11,10.0
